In [1]:
import numpy as np
import pandas as pd
from torch_geometric.data import Data

# PyTorch Geometric custom Dataset
This module illustrates the process part of the custom dataset creation for PyG done in the `datasets/SP100Stocks.py` file.
The Dataset class is used to create a custom dataset and use it with PyG modules.
**Note: To use this notebook, first execute notebooks 1 and 2.**

## Processing the graph data
The next cells demonstrate the adjacency matrix and historical values processing done in the `Dataset.process(*)` method.   

### Opening the historical values and the adjacency matrix
The goal is here to create T (the number of timestamps) graphs, one for each timestep. We use the same adjacency matrix for all the graphs as the structure is based on fundamentals and will not change.

In [2]:
values = pd.read_csv('../data/SP100/raw/values.csv').set_index(['Symbol', 'Date'])
values.head()

Close    Volume  NormClose  Variation  \
Symbol Date                                                                   
AAPL   2019-06-11 00:00:00-04:00  47.066402  0.195638  -1.911623   1.157954   
       2019-06-12 00:00:00-04:00  46.916603 -0.472454  -1.915051  -0.318272   
       2019-06-13 00:00:00-04:00  46.906937 -0.209103  -1.915272  -0.020603   
       2019-06-14 00:00:00-04:00  46.566280 -0.433329  -1.923068  -0.726239   
       2019-06-17 00:00:00-04:00  46.844124 -0.748329  -1.916710   0.596662   

                                       RSI      MACD  BhB  BlB  
Symbol Date                                                     
AAPL   2019-06-11 00:00:00-04:00  0.594733 -0.578354  0.0  0.0  
       2019-06-12 00:00:00-04:00  0.583426 -0.379584  0.0  0.0  
       2019-06-13 00:00:00-04:00  0.582656 -0.220298  0.0  0.0  
       2019-06-14 00:00:00-04:00  0.554870 -0.120165  0.0  0.0  
       2019-06-17 00:00:00-04:00  0.572766 -0.018180  0.0  0.0

In [3]:
adj = np.load('../data/SP100/raw/adj.npy')
adj[:5, :5]

array([[0.        , 0.        , 0.        , 0.25805462, 0.25805462],
       [0.        , 0.        , 0.51610923, 0.        , 0.        ],
       [0.        , 0.51610923, 0.        , 0.        , 0.        ],
       [0.25805462, 0.        , 0.        , 0.        , 0.51610923],
       [0.25805462, 0.        , 0.        , 0.51610923, 0.        ]])

### Creating the feature matrix and edge indices
The feature matrix is a 3d matrix of shape (timestamps_nb, stocks_nb, features_nb). The adjacency matrix is used to create the edge_index matrix in the PyTorch Geomtric format.

In [4]:
nodes_nb = len(adj)
x = np.array(
	values.drop(columns=["Close"]).to_numpy().reshape((nodes_nb, -1, values.shape[1] - 1))
)  # shape (nodes_nb, timestamps_nb, features_nb)
x = np.swapaxes(x, 1, 2)  # shape (nodes_nb, features_nb, timestamps_nb)

edge_nb = np.count_nonzero(adj)
edge_index = np.zeros((2, edge_nb))
edge_weight = np.zeros((edge_nb,))
count = 0
for i in range(nodes_nb):
	for j in range(nodes_nb):
		if (weight := adj[i, j]) != 0:
			edge_index[0, count], edge_index[1, count] = i, j
			edge_weight[count] = weight
			count += 1
x.shape, edge_index.shape, edge_weight.shape

((100, 7, 1234), (2, 2460), (2460,))

### Building the graphs
Finally, the graphs can be built using the Data class for PyG.

In [5]:
time_window = 25
timestamps = [
	Data(x=x[:, :, idx:idx+time_window], edge_index=edge_index, edge_weight=edge_weight, y=x[:, 1:2, idx+time_window]) for idx in range(x.shape[0] - time_window)
]
timestamps[:5]

[Data(x=[100, 7, 25], edge_index=[2, 2460], y=[100, 1], edge_weight=[2460]),
 Data(x=[100, 7, 25], edge_index=[2, 2460], y=[100, 1], edge_weight=[2460]),
 Data(x=[100, 7, 25], edge_index=[2, 2460], y=[100, 1], edge_weight=[2460]),
 Data(x=[100, 7, 25], edge_index=[2, 2460], y=[100, 1], edge_weight=[2460]),
 Data(x=[100, 7, 25], edge_index=[2, 2460], y=[100, 1], edge_weight=[2460])]

## Defining the PyTorch Geometric datasets
The previously defined method is used to process the data and create a standard PyG Dataset class.
The dataset contains graphs with 3d node features -- data for the timestamps $[\![t, t+T[\![$ -- and a target variable -- the variation at time $t+T$ --.

These dataset is used in the next notebooks. The code is in the `datasets/SP100Stocks.py` file.

In [6]:
from datasets.SP100Stocks import SP100Stocks

In [7]:
dataset = SP100Stocks()
dataset, dataset[0]

(SP100Stocks(1209),
 Data(x=[100, 7, 25], edge_index=[2, 2460], y=[100, 1], edge_weight=[2460], close_price=[100, 25], close_price_y=[100, 1]))